In [45]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [46]:
!pip install langchain==0.2.3
!pip install langchain-core==0.2.5
!pip install langchain-openai==0.2.0
!pip install python-dotenv
!pip install langgraph
!pip install langchain-google-genai

  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
Using cached langchain_core-0.2.43-py3-none-any.whl (397 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.5 requires langchain-core<0.4.0,>=0.3.62, but you have langchain-core 0.2.43 which is incompatible.
langchain-openai 0.2.0 requires langchain-core<0.4,>=0.3, but you have langchain-core 0.2.43 which is incompatible.
langgraph-prebuilt 0.2.2 requires langchain-core>=0.3.22, but you have langchain-core 0.2.43 which is incompatible.
  Using cached langchain_core-0.2.5-py3-none-any.whl.metadata (5.8 kB)
Using cached langchain_core-0.2.5-py3-none-any.whl (314 kB)
  Attempting un

In [47]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

In [48]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GEMINI_KEY")

In [49]:
#global variable
document_content = ""

In [50]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]


@tool
def update(content: str) -> str:
    """Updates the document with the provided text"""
    global document_content
    document_content = content
    return f"Document has been updated successfully! The current content is: \n{document_content}"

In [51]:
@tool
def save(filename: str) -> str:
    """save the current document to a text file and finish the process"""

    global document_document_content 
    
    if not filename.endswith('.txt'):
        filename = f"{filename.txt}"

    try:
            with open(filename, 'w') as file:
                file.write (document_content)

            print(f"Document has been saved to {filename}")
            return f"Document has been saved to '{filename}'"

    except Exception as e:
        return f"Error saving document: {str(e)}"

tools = [update, save]

In [52]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7, google_api_key=secret_value_0).bind_tools(tools)

In [53]:
def our_agent(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(content=f"""
    You are Drafter, a helpful writing assistant. You are going to help the user update and modify documents.
    
    - If the user wants to update or modify content, use the 'update' tool with the complete updated content.
    - If the user wants to save and finish, you need to use the 'save' tool.
    - Make sure to always show the current document state after modifications.
    
    The current document content is:{document_content}
    """)

    if not state["messages"]:
        user_input = "I'm ready to help you update a document. What would you like to create?"
        user_message = HumanMessage(content=user_input)
    else:
        user_input = input("\nWhat would you like to do with the document? ")
        print(f"\n👤 USER: {user_input}")
        user_message = HumanMessage(content=user_input)

    all_messages = [system_prompt] + list(state["messages"]) + [user_message]

    try:
        response = model.invoke(all_messages)
        print(f"\n🤖 AI: {response.content}")
        if hasattr(response, "tool_calls") and response.tool_calls:
            print(f"🔧 USING TOOLS: {[tc['name'] for tc in response.tool_calls]}")
        return {"messages": list(state["messages"]) + [user_message, response]}
    except Exception as e:
        print(f"❌ Error while invoking model: {e}")
        return {"messages": list(state["messages"]) + [user_message]}


In [54]:
def should_continue(state: AgentState) -> str:
    """Determine if we should continue or end the conversation"""
    messages = state["messages"]
    if not messages:
        return "continue"

    # This looks for the most recent tool message....
    for message in reversed(messages):
        # ... and checks if this is a ToolMessage resulting from save
        if (isinstance(message, ToolMessage) and 
            "saved" in message.content.lower() and
            "document" in message.content.lower()):
            return "end" # goes to the end edge which leads to the endpoint
        
    return "continue"

In [55]:
def print_messages(messages):
    """Function made to print the messages in a more reaable format"""
    if not messages:
        return
    for message in messages[-3:]:
        if isinstance(message, ToolMessage):
            print(f"\n TOOL RESULT: {message.content}")

In [56]:
graph = StateGraph(AgentState)

graph.add_node("agent", our_agent)
graph.add_node("tools", ToolNode(tools))

graph.set_entry_point("agent")

graph.add_edge("agent", "tools")


graph.add_conditional_edges(
    "tools",
    should_continue,
    {
        "continue": "agent",
        "end": END,
    },
)

app = graph.compile()

In [57]:
def run_document_agent():
    print("\n ===== DRAFTER =====")
    
    state = {"messages": []}
    
    for step in app.stream(state, stream_mode="values"):
        if "messages" in step:
            print_messages(step["messages"])
    
    print("\n ===== DRAFTER FINISHED =====")


In [58]:
run_document_agent()


 ===== DRAFTER =====

🤖 AI: I'd like to create a document that outlines the benefits of using electric vehicles.  Let's start with a simple introduction.



What would you like to do with the document?  write me an emailto tom saying i will not be able to make it to the meeting



👤 USER: write me an emailto tom saying i will not be able to make it to the meeting

🤖 AI: 
🔧 USING TOOLS: ['update']

 TOOL RESULT: Document has been updated successfully! The current content is: 
Subject: Unable to Attend Meeting

Hi Tom,

I am writing to inform you that I will unfortunately be unable to attend the meeting today.  I apologize for any inconvenience this may cause.

[Optional: Briefly explain reason, if appropriate.  Keep it concise.]

Thanks,

[Your Name]



What would you like to do with the document?  make sure to specify the meeting time which is 10.00 a.m. and i had another meeting on that time



👤 USER: make sure to specify the meeting time which is 10.00 a.m. and i had another meeting on that time

🤖 AI: 
🔧 USING TOOLS: ['update']

 TOOL RESULT: Document has been updated successfully! The current content is: 
Subject: Unable to Attend Meeting

Hi Tom,

I am writing to inform you that I will unfortunately be unable to attend the meeting today.  I apologize for any inconvenience this may cause.

[Optional: Briefly explain reason, if appropriate.  Keep it concise.]

Thanks,

[Your Name]

 TOOL RESULT: Document has been updated successfully! The current content is: 
Subject: Unable to Attend Meeting at 10:00 AM

Hi Tom,

I am writing to inform you that I will unfortunately be unable to attend our meeting at 10:00 AM today. I have another meeting scheduled at the same time. I apologize for any inconvenience this may cause.

Thanks,

[Your Name]


...


👤 USER: my name in Amalka

🤖 AI: 
🔧 USING TOOLS: ['update']

 TOOL RESULT: Document has been updated successfully! The current content is: 
Subject: Unable to Attend Meeting at 10:00 AM

Hi Tom,

I am writing to inform you that I will unfortunately be unable to attend our meeting at 10:00 AM today. I have another meeting scheduled at the same time. I apologize for any inconvenience this may cause.

Thanks,

[Your Name]

 TOOL RESULT: Document has been updated successfully! The current content is: 
Subject: Unable to Attend Meeting at 10:00 AM

Hi Tom,

I am writing to inform you that I will unfortunately be unable to attend our meeting at 10:00 AM today. I have another meeting scheduled at the same time. I apologize for any inconvenience this may cause.

Thanks,

Amalka



What would you like to do with the document?  tell him that i will be able to meet him around 1.00 p.m. on the next day



👤 USER: tell him that i will be able to meet him around 1.00 p.m. on the next day

🤖 AI: 
🔧 USING TOOLS: ['update']

 TOOL RESULT: Document has been updated successfully! The current content is: 
Subject: Unable to Attend Meeting at 10:00 AM

Hi Tom,

I am writing to inform you that I will unfortunately be unable to attend our meeting at 10:00 AM today. I have another meeting scheduled at the same time. I apologize for any inconvenience this may cause.

Thanks,

Amalka

 TOOL RESULT: Document has been updated successfully! The current content is: 
Subject: Unable to Attend Meeting at 10:00 AM

Hi Tom,

I am writing to inform you that I will unfortunately be unable to attend our meeting at 10:00 AM today. I have another meeting scheduled at the same time. I apologize for any inconvenience this may cause.

I am available to meet tomorrow around 1:00 PM if that works for you.

Thanks,

Amalka



What would you like to do with the document?  i like this. save it please



👤 USER: i like this. save it please

🤖 AI: 
🔧 USING TOOLS: ['save']

 TOOL RESULT: Document has been updated successfully! The current content is: 
Subject: Unable to Attend Meeting at 10:00 AM

Hi Tom,

I am writing to inform you that I will unfortunately be unable to attend our meeting at 10:00 AM today. I have another meeting scheduled at the same time. I apologize for any inconvenience this may cause.

I am available to meet tomorrow around 1:00 PM if that works for you.

Thanks,

Amalka
Document has been saved to meeting_apology.txt

 TOOL RESULT: Document has been saved to 'meeting_apology.txt'

 ===== DRAFTER FINISHED =====
